<a href="https://colab.research.google.com/github/JhonHader/HAND-KEYPOINTS-DETECTION/blob/main/Hand_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <strong>Hand Keypoints Detection</strong>

*   **<font color='red'> Description problem </font>** 
##### Hand-detection and its keypoints.

---
---

#####Developed by: 
<h6 align=center> ${\text{Jhon Hader Fernández}}$ <h6>
<h6 align=center> ${\text{Diego Fernando Díaz}}$ <h6>

#####<h6 align=center>{<i>jhon_fernandez, di-diego</i>}@javeriana.edu.co<h6>
#####<h6 align=center>Pontificia Universidad Javeriana<h6>

<br>

## ***1. ENVIRONMENT***

To use this project is recommended use GPU execution enviroment, to this is so important install all NVIDIA-CUDA dependences (to run deep neural networks in OpenCV).
This could take a long, therefore we've created a enviroment, to use you have to get its link (it's located in Drive)

In [ ]:
install_GPU_dependencies = False

if install_GPU_dependencies == True:
  %cd /content
  !git clone https://github.com/opencv/opencv
  !git clone https://github.com/opencv/opencv_contrib
  !mkdir /content/build
  %cd /content/build

  !cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF  -DBUILD_EXAMPLES=OFF  -DWITH_OPENEXR=OFF  -DWITH_CUDA=ON  -DWITH_CUBLAS=ON  -DWITH_CUDNN=ON  -DOPENCV_DNN_CUDA=ON  /content/opencv

  !make -j8 install

  !mkdir  "/content/drive/My Drive/cv2_cuda"
  !cp  /content/build/lib/python3/cv2.cpython-36m-x86_64-linux-gnu.so "/content/drive/My Drive/cv2_cuda"

else:
  print('[INFO...] CUDA-GPU and OpenCV dependences requirements already installed!')


### ***1.1. GET ENVIRONMENT***

* **<font color='green'><i> 1.1.1. </i></font>** <br>
Get enviroment from Google Drive, please chechk your execution environment, it has to be configurated in **<font color='red'><i> GPU </i></font>**

* **<font color='green'><i> 1.1.2. </i></font>** <br>
Import libraries 
<br>


**<font color='red'><i> REQUIREMENT </i></font>** <br>
OpenCV version has to be greater than **<font color='blue'><i> 4.2.x </i></font>**

In [ ]:
GPU = True
if GPU == True:
  print('[INFO...] Loading GPU dependences.')
  !cp "/content/drive/My Drive/cv2_cuda/cv2.cpython-36m-x86_64-linux-gnu.so" .
  print('\n[INFO...] GPU dependendes was loaded, OpenCV version has to be greater 4.2.x')

import cv2
from __future__ import division
import time
import numpy as np
from skimage import exposure
import time
import os
import sys
from google.colab.patches import cv2_imshow
from IPython.display import clear_output
print('[INFO...] OpenCV version:', cv2.__version__)

## ***2. GET VIDEO STREAM***

You're running code in a cloud server, that isn't your hardware, then, to access to your camera is neccesary use API. This snippet access to your camera and record video stream.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def record_video(filename='video.mp4'):
  js = Javascript("""
    async function recordVideo() {
      // mashes together the advanced_outputs.ipynb function provided by Colab, 
      // a bunch of stuff from Stack overflow, and some sample code from:
      // https://developer.mozilla.org/en-US/docs/Web/API/MediaStream_Recording_API

      // Optional frames per second argument.
      const options = { mimeType: "video/webm; codecs=vp9" };
      const div = document.createElement('div');
      const capture = document.createElement('button');
      const stopCapture = document.createElement("button");
      capture.textContent = "Start Recording";
      capture.style.background = "green";
      capture.style.color = "white";

      stopCapture.textContent = "Stop Recording";
      stopCapture.style.background = "red";
      stopCapture.style.color = "white";
      div.appendChild(capture);

      const video = document.createElement('video');
      const recordingVid = document.createElement("video");
      video.style.display = 'block';

      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      // create a media recorder instance, which is an object
      // that will let you record what you stream.
      let recorder = new MediaRecorder(stream, options);
      document.body.appendChild(div);
      div.appendChild(video);
      // Video is a media element.  This line here sets the object which serves
      // as the source of the media associated with the HTMLMediaElement
      // Here, we'll set it equal to the stream.
      video.srcObject = stream;
      // We're inside an async function, so this await will fire off the playing
      // of a video. It returns a Promise which is resolved when playback has 
      // been successfully started. Since this is async, the function will be 
      // paused until this has started playing. 
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      // and now, just wait for the capture button to get clicked in order to
      // start recording
      await new Promise((resolve) => {
        capture.onclick = resolve;
      });
      recorder.start();
      capture.replaceWith(stopCapture);
      // use a promise to tell it to stop recording
      await new Promise((resolve) => stopCapture.onclick = resolve);
      recorder.stop();

      let recData = await new Promise((resolve) => recorder.ondataavailable = resolve);
      let arrBuff = await recData.data.arrayBuffer();
      
      // stop the stream and remove the video element
      stream.getVideoTracks()[0].stop();
      div.remove();

      let binaryString = "";
      let bytes = new Uint8Array(arrBuff);
      bytes.forEach((byte) => {
        binaryString += String.fromCharCode(byte);
      })
      return btoa(binaryString);
    }
    """)
  try:
    display(js)
    data = eval_js('recordVideo({})')
    binary = b64decode(data)
    with open(filename, "wb") as video_file:
      video_file.write(binary)
    print(
        f"Finished recording video. Saved binary under filename in current working directory: {filename}"
    )
  except Exception as err:
      # In case any exceptions arise
      print(str(err))
  return filename

### ***2.1. RECORD VIDEO STREAM***

Record a video stream, this recording is saved in local enviroment, when you close the session that video will be deleted and all data in local environment too.

In [ ]:
input_recording = 'mirror_test.avi'     # filename to save recording, you should save it in '.avi' format
record_video(filename=input_recording)

### ***2.2. PLAY VIDEO RECORDED***

Play video stream recorded.

In [ ]:
from IPython.display import HTML
from base64 import b64encode

video_width = 800
video_file = open(input_recording, "r+b").read()
video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

## ***3. GET FRAMES OF VIDEO***

Get all frames in the video.

In [ ]:
cap = cv2.VideoCapture(input_recording)

fps = cap.get(cv2.CAP_PROP_FPS)
(width, height) = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

print(fps, (width, height))

video = []

print('[INFO...] Getting frames of video.')
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
      break
    video.append(frame)
print('[REPORT...] Frames of video was obteined succesfully.')

## ***4. HAND KEYPOINTS DETECTION***

Get all keypoints detection for every frame in the video.

In [ ]:
root_path = '/content/drive/My Drive/Hand_Detection'

protoFile = os.path.join(root_path, "MODELS/pose_deploy.prototxt.txt")
weightsFile = os.path.join(root_path, "MODELS/pose_iter_102000.caffemodel")
nPoints = 22
POSE_PAIRS = [[0, 1], [1, 2], [2, 3], [3, 4], [0, 5], [5, 6], [6, 7], [7, 8], [0, 9],
              [9, 10], [10, 11], [11, 12], [0, 13], [13, 14], [14, 15], [15, 16],
              [0, 17], [17, 18], [18, 19], [19, 20]]

skeleton = []

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
if GPU == True:
  print('[INFO...] GPU was configurated!')
  net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
  net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

frameWidth = width
frameHeight = height
aspect_ratio = frameWidth / frameHeight

threshold = 0.1

inHeight = 368
inWidth = int(((aspect_ratio * inHeight) * 8) // 8)

t_total = time.time()

print('[INFO...] Processing video, getting hand keypoints!')
for frame in video: 

  points = []

  img_original = frame.copy()
  frameCopy = np.copy(frame)

  inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                                  (0, 0, 0), swapRB=False, crop=False)

  net.setInput(inpBlob)
  output = net.forward()
  for i in range(nPoints):

      probMap = output[0, i, :, :]
      probMap = cv2.resize(probMap, (frameWidth, frameHeight))

      minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

      if prob > threshold:
        points.append((int(point[0]), int(point[1])))
      else:
        points.append(None)

  for pair in POSE_PAIRS:
      partA, partB = pair[0], pair[1]

      if points[partA] and points[partB]:
          cv2.line(frame, points[partA], points[partB], (0, 0, 255), 2)
          cv2.circle(frame, points[partA], 4, (0, 255, 0), thickness=-1, lineType=cv2.FILLED)
          cv2.circle(frame, points[partB], 4, (0, 255, 0), thickness=-1, lineType=cv2.FILLED)

  skeleton.append(frame)
print("\n[REPORT...] Hand keypoints was gotten successfully!")
print("[REPORT...] time taken processing video [s] : {:.3f}".format(time.time() - t_total))

## ***5. SHOW KEYPOINTS***

Show keypoints for every frame in the video.

In [ ]:
%pylab inline
import matplotlib.pyplot as plt

for frame in skeleton:
  #plt.figure(figsize = (height*0.015, width*0.015))
  plt.imshow(frame)
  plt.show()
  time.sleep(1//6.25)
  clear_output(wait=True)